# Linked Art API 协议

> **上级页面**: ["Linked Art API 1.0"]("/api/1.0/")

---



In [ ]:
# 环境设置
from cromulent import model, vocab

# 设置 base_url 以获得更清晰的输出
model.factory.base_url = 'http://test.com/museum/'





## 简介

API 的目的是提供一个应用程序开发人员可以用来与系统中数据交互的接口，该系统在某种程度上与他们正在实施的系统分离。在某些情况下，这种分离可能只是在一个代码库内，不同的方面由不同的软件工程师开发。在 Linked Art API 中，这种分离更加具体——消费应用程序的代码与发布系统的代码完全分离，我们需要一个协议在系统之间通过网络传输请求和响应。

[关联开放可用数据](/loud/) 承认这种网络交互有成本，各种软件工程师之间的社会分离增加了这些成本。用于将数据从发布者传输到消费应用程序的协议需要尽可能最小化和易于理解，以避免进一步复杂化实施。因此，我们更喜欢以资源为中心的接口（或"REST"）的策略，而不是面向服务的架构。我们使用网络作为核心超媒体平台，因此使用 HTTP 作为交互协议。

使用 REST 范式和 HTTP 协议遵循我们的[设计原则](../principles/)。本文档定义了在此上下文中如何解释 REST 的具体细节，以及实施应该考虑 HTTP 的哪些功能。

## HTTP 实施细节

### HTTPS 和协议版本

强烈建议发布实施使用 HTTP 的加密形式，称为 HTTPS。即使对于完全开放和可重用的数据，安全性也保护用户隐私，并允许实施模式，其中数据不是完全开放的，而是需要某种程度的身份验证。端点的 URI，因此被描述的实体，将使用 `https` 协议方案。

例如，本文档使用 https，产生如下端点：

> `https://linked.art/example/object/1`

支持 HTTP 协议的最新版本在网络交互速度方面有很大优势。支持 HTTP 2 及以上允许多个并发请求进行，改善完成页面加载的时间，并避免浏览器对每个站点并发连接数的限制。Web 服务器必须支持 HTTP 1.1，应该支持 HTTP 2。

### 操作

API 1.0 版本支持的唯一操作是从端点检索表示。用 HTTP 术语来说，这意味着只直接使用 `GET` 方法，不使用 `POST`（用于创建资源）、`PUT`（用于更新资源）或 `DELETE`（用于删除资源）。

但是，还有两个其他操作需要考虑。服务器必须支持 `OPTIONS` 请求，以允许在 javascript 中与浏览器交互，这个过程称为 CORS。这在下面有自己的部分描述。其次，有 `HEAD` 方法，它返回关于资源的标头信息，而不包括实际的表示。支持这个方法很有用，但消费客户端绝不能依赖它。

发布者必须支持 GET 和 OPTIONS，应该支持 HEAD。


### HTTP 标头

为了允许端点的 URI 与其他系统和表示共存，需要向端点发送一个标头，指定所需的格式。这通常称为"内容协商"，有时只是"conneg"。所需格式的媒体类型在 `Accept` 标头中发送，该媒体类型（根据 [json-ld](../json-ld/) 文档）是：

> `Accept: application/ld+json;profile="https://linked.art/ns/v1/linked-art.json"`

然后服务器将以正确的格式响应，并在 `Content-Type` 标头中回传媒体类型：

> `Content-Type: application/ld+json;profile="https://linked.art/ns/v1/linked-art.json"`

服务器也可能支持其他序列化以及 Linked Art JSON-LD 在相同的 URI，这将通过更改 `Accept` 标头中给出的媒体类型来请求。

Linked Art 没有指定一种机制来发现其他支持格式的集合。

在未来，模型和 API 的新主版本将具有不同的配置文件 URI，从而允许应用程序使用规范的不同版本请求相同的实体。

### CORS

为了允许基于浏览器的应用程序与数据交互，发布者必须确保跨域资源共享或 [CORS](https://developer.mozilla.org/en-US/docs/Web/HTTP/CORS) 标头正确设置。特别是，响应上的 `Access-Control-Allow-Origin` 标头必须在每个响应上设置为"*"。

> `Access-Control-Allow-Origin: *`

资源还必须响应 `OPTIONS` 请求（称为预检请求）的此标头，以允许数据跨系统使用，并使用非常普通的媒体类型。


## URI 良好实践

对于 Linked Art 的任何端点或实例的 URI 没有要求，因为 URI 永远不需要被操作来使服务行为不同。通过不要求特定模式，我们启用许多不同的实施，包括重用具有不同格式的现有 URI。

但是，承认在没有任何其他需要考虑的情况下，对于 URI 的构建有一些良好的实践。这些分为 URI 的一般理想功能和 URI 结构本身内的模式。

### 理想功能

__URI 应该尽可能持久。__ 应该避免 URI 中任何可能需要在未来更改的内容，包括特定的技术引用、无法维护的技术特定标识符、可能被重命名的机构部门等等。

__URI 应该简短且可读，在合理范围内。__ 较短的 URI 通常比较长的更受欢迎，因为它们更令人难忘，更容易复制或输入。因此，Linked Art API URI 应该能够简短且可读……在合理范围内，考虑到它们几乎肯定通过数据库由自动化系统管理，持久性比可读性更重要。

__URI 应该启用合理的数据管理和简单的实施。__ 由于实施应该能够通过将文件放在网络服务器上的磁盘上来实现，URI 必须能够反映这种策略而不会造成不当负担。

__URI 仍必须被视为不透明字符串。__ 不管 URI 的结构和组件的上述功能如何，它们仍必须被视为完全包含无用信息的不透明字符串。编码模式的存在是为了促进软件工程师理解 API，而不是为了促进软件与其交互。


### 首选 URI 结构

满足上述理想需求的首选结构是：

> scheme `://` hostname `/` _prefix_ `/` endpoint `/` identifier

其中

* `scheme` 如果可能为"https"，如果不可能则为"http"
* `hostname` 如果可能为拥有所描述资源的机构的域名，否则为管理数据的域名
* `prefix` 是可选的路径组件，可能包括 / 字符，以允许数据在网站层次的任何级别发布
* `endpoint` 是每个端点的固定路径组件，用于将给定类型的所有条目收集在一起
* `identifier` 是条目的唯一标识符

因此一个例子可能是：

> `https://museum.org/data/object/opaque-identifier`


首选端点路径名称：

* `concept` - [概念](../endpoint/concept/)
* `digital` - [数字对象](../endpoint/digital_object/)
* `event` - [事件](../endpoint/event/)
* `group` - [群体](../endpoint/group/)
* `object` - [物理对象](../endpoint/physical_object/)
* `person` - [个人](../endpoint/person/)
* `place` - [地点](../endpoint/place/)
* `provenance` - [流传历史活动](../endpoint/provenance_activity/)
* `set` - [藏品集合和集合](../endpoint/set/)
* `text` - [文本作品](../endpoint/textual_work/)
* `visual` - [视觉作品](../endpoint/visual_work/)